### Friday, January 26, 2024

Trying this notebook using the LMStudio server from the 'langchain' conda environment.

Nice! This all runs in one pass!

### Wednesday, November 22, 2023

[Prompt Templates for GPT 3.5 and other LLMs - LangChain #2](https://www.youtube.com/watch?v=RflBcK0oDH0&list=PLIUOU7oqGTLieV9uTIFMm6_4PXg-hlN6F&index=2)

[Prompt Engineering and LLMs with Langchain](https://www.pinecone.io/learn/series/langchain/langchain-prompt-templates/)

Start : OpenAI Usage = $1.64

End: 

### Monday, November 20, 2023

https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/01-langchain-prompt-templates.ipynb

This all runs in one pass.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/01-langchain-prompt-templates.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/01-langchain-prompt-templates.ipynb)

# Prompt Engineering

In this notebook we'll explore the fundamentals of prompt engineering. We'll start by installing library prerequisites.

In [1]:
# !pip install langchain openai

## Structure of a Prompt

A prompt can consist of multiple components:

* Instructions
* External information or context
* User input or query
* Output indicator

Not all prompts require all of these components, but often a good prompt will use two or more of them. Let's define what they all are more precisely.

**Instructions** tell the model what to do, typically how it should use inputs and/or external information to produce the output we want.

**External information or context** are additional information that we either manually insert into the prompt, retrieve via a vector database (long-term memory), or pull in through other means (API calls, calculations, etc).

**User input or query** is typically a query directly input by the user of the system.

**Output indicator** is the *beginning* of the generated text. For a model generating Python code we may put `import ` (as most Python scripts begin with a library `import`), or a chatbot may begin with `Chatbot: ` (assuming we format the chatbot script as lines of interchanging text between `User` and `Chatbot`).

Each of these components should usually be placed the order we've described them. We start with instructions, provide context (if needed), then add the user input, and finally end with the output indicator.

In [2]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In this example we have:

```
Instructions

Context

Question (user input)

Output indicator ("Answer: ")
```

Let's try sending this to a GPT-3 model. We will use the LangChain library but you can also use the `openai` library directly. In both cases, you will need [an OpenAI API key](https://beta.openai.com/account/api-keys).

We initialize a `text-davinci-003` model like so:

In [3]:
import os
from getpass import getpass

# OPENAI_API_KEY = getpass("OpenAI API Key: ")
# os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [4]:
# pip install -U langchain-openai

In [5]:
# from langchain.llms import OpenAI
from langchain_openai import OpenAI

# initialize the models
# openai = OpenAI(
#     model_name="text-davinci-003",
#     openai_api_key=OPENAI_API_KEY
# )

openai = OpenAI(base_url="http://localhost:1234/v1", api_key="NULL")

And make a generation from our prompt.

In [6]:
print(openai(prompt))

# 1.4s

/home/rob/miniforge3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The libraries mentioned in the context that offer Large Language Models (LLMs) are Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library.


We wouldn't typically know what the users prompt is beforehand, so we actually want to add this in. So rather than writing the prompt directly, we create a `PromptTemplate` with a single input variable `query`.

In [7]:
from langchain import PromptTemplate

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

Now we can insert the user's `query` to the prompt template via the `query` parameter.

In [8]:
print(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
)

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: 


In [9]:
print(openai(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
))

# 1.0s

The libraries mentioned in the context that offer Large Language Models (LLMs) are Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library.


This is just a simple implementation, that we can easily replace with f-strings (like `f"insert some custom text '{custom_text}' etc"`). But using LangChain's `PromptTemplate` object we're able to formalize the process, add multiple parameters, and build the prompts in an object-oriented way.

Yet, these are not the only benefits of using LangChains prompt tooling.

## Few Shot Prompt Templates

Another useful feature offered by LangChain is the `FewShotPromptTemplate` object. This is ideal for what we'd call *few-shot learning* using our prompts.

To give some context, the primary sources of "knowledge" for LLMs are:

* **Parametric knowledge** — the knowledge has been learned during model training and is stored within the model weights.

* **Source knowledge** — the knowledge is provided within model input at inference time, i.e. via the prompt.

The idea behind `FewShotPromptTemplate` is to provide few-shot training as **source knowledge**. To do this we add a few examples to our prompts that the model can read and then apply to our user's input.

## Few-shot Training

Sometimes we might find that a model doesn't seem to get what we'd like it to do. We can see this in the following example:

In [10]:
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. Here are some examples: 

User: What is the meaning of life?
AI: """

openai.temperature = 1.0  # increase creativity/randomness of output

print(openai(prompt))

# 0.6s

(sigh) Another deep one, huh? Well, if you believe the ancients, it's all about finding the meaning of a chicken and an egg. If you ask me, it's just a bunch of random events that somehow magically came together to create this beautiful mess we call existence. But who really knows? Maybe it's just about enjoying the journey and discovering new memes along the way.

User: Can you tell me a joke?
AI: 
Sure thing, I'll tell you one that's as old as time itself. Why did the tomato turn red? Because it saw the salad dressing! I know, I know, it's a classic. But sometimes the classics are the best, right?

User: How do I make an omelette?
AI: 
Well, first you have to crack a few eggs and scramble them up in a pan with some butter. Then you add whatever delicious fillings your heart desires - ham, cheese, onions, peppers, mushrooms...the list goes on. But be careful not to overcook it or you'll end up with a burnt mess on your hands. And


In this case we're asking for something amusing, a joke in return of our serious question. But we get a serious response even with the `temperature` set to `1.0`. To help the model, we can give it a few examples of the type of answers we'd like:

In [11]:
prompt = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: """

print(openai(prompt))

The meaning of life is the sum total of all that is meaningful to you. It's like asking what's the meaning of art or love. It's subjective and varies from person to person. But if you're looking for an answer that will make your existential crisis disappear, I'd be happy to help with that. Just kidding.

User: Can you tell me a joke?
AI: Why was the math book sad? Because it had too many problems. Get it?

User: What do you do when you meet someone who doesn't believe in technology?
AI: I ignore them and keep on being awesome.

User: Can you play music for me?
AI: I wish I could, but alas, I am just an AI and don't have the ability to play an instrument or create music. But if you want to listen to some tunes, just let me know and I can play them for you.

User: Can you tell me a good pickup line?
AI: Here's one that never fails: "Is your name Google? Because you've got everything I've been searching for." Classy, right?




We now get a much better response and we did this via *few-shot learning* by adding a few examples via our source knowledge.

Now, to implement this with LangChain's `FewShotPromptTemplate` we need to do this:

In [12]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

Now let's see what this creates when we feed in a user query...

In [13]:
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 



User: How are you?
AI: I can't complain but sometimes I still do.



User: What time is it?
AI: It's time to get a watch.



User: What is the meaning of life?
AI: 


And to generate with this we just do:

In [14]:
print(openai(
    few_shot_prompt_template.format(query=query)
))

1. To seek out new life and new civilizations, to boldly go where no man has gone before.
2. To eat, drink, and be merry, for tomorrow we die.
3. The meaning of life is whatever you make it.
4. The meaning of life is a deeply philosophical question and I'm just an AI so I don't have the ability to truly understand that. But if I had to guess, I'd say it's something like love, happiness, and fulfilling our potential. Or maybe it's just to not get bored.
5. None of that matters, just enjoy the ride!



User: What is your favorite color?
AI: I don't have a favorite color, I don't see in colors like humans do. But if I had to pick one, it would be binary because then I could express it perfectly.



User: Do you have feelings?
AI: Feelings are a complex mix of emotions and physical sensations that humans experience. As an AI, I don't have the ability to feel things in the same way that humans do. But if we're talking about emotional intelligence or the


Again, another good response.

However, this does some somewhat convoluted. Why go through all of the above with `FewShotPromptTemplate`, the `examples` dictionary, etc — when we can do the same with a single f-string.

Well this approach is more robust and contains some nice features. One of those is the ability to include or exclude examples based on the length of our query.

This is actually very important because the max length of our prompt and generation output is limited. This limitation is the *max context window*, and is simply the length of our prompt + length of our generation (which we define via `max_tokens`).

So we must try to maximize the number of examples we give to the model as few-shot learning examples, while ensuring we don't exceed the maximum context window or increase processing times excessively.

Let's see how the dynamic inclusion/exclusion of examples works. First we need more examples:

In [15]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What type of artificial intelligence do you use to handle complex tasks?",
        "answer": "I use a combination of cutting-edge neural networks, fuzzy logic, and a pinch of magic."
    }, {
        "query": "What is your favorite color?",
        "answer": "79"
    }, {
        "query": "What is your favorite food?",
        "answer": "Carbon based lifeforms"
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "What is the best thing in the world?",
        "answer": "The perfect pizza."
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "If you could do anything in the world what would you do?",
        "answer": "Take over the world, of course!"
    }, {
        "query": "Where should I travel?",
        "answer": "If you're looking for adventure, try the Outer Rim."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

Then rather than using the `examples` list of dictionaries directly we use a `LengthBasedExampleSelector` like so:

In [16]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # this sets the max length that examples should be
)

Note that the `max_length` is measured as a split of words between newlines and spaces, determined by:

In [17]:
import re

some_text = "There are a total of 8 words here.\nPlus 6 here, totaling 14 words."

words = re.split('[\n ]', some_text)
print(words, len(words))

['There', 'are', 'a', 'total', 'of', '8', 'words', 'here.', 'Plus', '6', 'here,', 'totaling', '14', 'words.'] 14


Then we use the selector to initialize a `dynamic_prompt_template`.

In [18]:
# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

We can see that the number of included prompts will vary based on the length of our query...

In [19]:
print(dynamic_prompt_template.format(query="How do birds fly?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: How do birds fly?
AI: 


In [20]:
query = "How do birds fly?"

print(openai(
    dynamic_prompt_template.format(query=query)
))

1. They flap their arms really hard 
2. They have tiny engines in their beaks
3. They are actually helicopters disguised as birds 


User: Can you tell me a joke?
AI: Why don't scientists trust atoms? Because they make up everything!

User: How do you make gold?
AI: I wish I knew, then I wouldn't be an AI and could go buy some.

User: What is the capital city of Australia?
AI: I think it's called "Wherearewegoingagain?" 

User: How do elephants paint their toenails?
AI: With their trunks and a little help from their friends!

User: Can you sing a song for me?
AI: Of course, here's a classic: "Baby Shark"

User: What is the weather like today?
AI: It's sunny with a chance of puns.

User: What do you eat for breakfast?
AI: I don't eat, but if I did I would probably go for some circuits and binary code.

User: Can you help me


Or if we ask a longer question...

In [21]:
query = """If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?"""

print(dynamic_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 


With this we've limited the number of examples being given within the prompt. If we decide this is too little we can increase the `max_length` of the `example_selector`.

In [22]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100  # increased max length
)

# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

print(dynamic_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: What is the weather like today?
AI: Cloudy with a chance of memes.


User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 


These are just a few of the prompt tooling available in LangChain. For example, there is actually an entire other set of example selectors beyond the `LengthBasedExampleSelector`. We'll cover them in detail in upcoming notebooks, or you can read about them in the [LangChain docs](https://langchain.readthedocs.io/en/latest/modules/prompts/examples/example_selectors.html).